In [1]:
import json
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
import bbox_visualizer as bbv
from PIL import Image
import numpy as np
from IPython.display import display
import pandas as pd
import random

In [2]:
def make_coco_dataset(json_path, path_to_images=None, encoding='utf-8'):
    
    with open(json_path, encoding=encoding) as cocojson:
        annotations_json = json.load(cocojson)
        
    # Store the 3 sections of the json as seperate json arrays
    images = pd.json_normalize(annotations_json["images"])
    images.columns = "img_" + images.columns
    try:
        images["img_folder"]
    except:
        images["img_folder"] = ""
    
    # If specified a different image folder then use that one
    if path_to_images != None:
        images["img_folder"] = path_to_images

    astype_dict = {"img_width": "int64", "img_height": "int64", "img_depth": "int64"}
    astype_keys = list(astype_dict.keys())
    for element in astype_keys:
        if element not in images.columns:
            astype_dict.pop(element)
            
    images = images.astype(astype_dict)

    annotations = pd.json_normalize(annotations_json["annotations"])
    annotations.columns = "ann_" + annotations.columns

    categories = pd.json_normalize(annotations_json["categories"])
    categories.columns = "cat_" + categories.columns
    categories.cat_id = categories.cat_id.astype(str)
    
    df = annotations

    # Converting this to string resolves issue #23
    df.ann_category_id = df.ann_category_id.astype(str)

    df[
        ["ann_bbox_xmin", "ann_bbox_ymin", "ann_bbox_width", "ann_bbox_height"]
    ] = pd.DataFrame(df.ann_bbox.tolist(), index=df.index)
    df.insert(8, "ann_bbox_xmax", df["ann_bbox_xmin"] + df["ann_bbox_width"])
    df.insert(10, "ann_bbox_ymax", df["ann_bbox_ymin"] + df["ann_bbox_height"])

    # debug print(df.info())

    # Join the annotions with the information about the image to add the image columns to the dataframe
    df = pd.merge(images, df, left_on="img_id", right_on="ann_image_id", how="left")
    df = pd.merge(
        df, categories, left_on="ann_category_id", right_on="cat_id", how="left"
    )

    # Rename columns if needed from the coco column name to the pylabel column name
    df.rename(columns={"img_file_name": "img_filename"}, inplace=True)
    
    df.fillna("", inplace=True)

    # These should be strings
    df.cat_id = df.cat_id.astype(str)

    # These should be integers
    df.img_width = df.img_width.astype(int)
    df.img_height = df.img_height.astype(int)
    
    return df

In [9]:
coco_json_path = '/home/ubuntu/workspace/datasets/dacon/coco/annotations/train_sample_10.json'
coco_image_path = '/home/ubuntu/workspace/datasets/dacon/coco/train'

coco_dataset = make_coco_dataset(coco_json_path, coco_image_path)
coco_dataset.head(1)

,img_id,img_folder,img_filename,img_path,img_width,img_height,img_depth,ann_image_id,ann_id,ann_segmented,...,ann_bbox_ymax,ann_category_id,ann_difficult,ann_bbox_xmin,ann_bbox_ymin,ann_bbox_width,ann_bbox_height,cat_id,cat_name,cat_supercategory
0,0,/home/ubuntu/workspace/datasets/dacon/coco/train,syn_00198.png,,1920,1040,3,0,0,,...,742,26,,1131,391,336,351,26,kia_sorrento_suv_2020_,none


In [ ]:
def visualize_random_class_samples(dataset, images_per_class, specific_class):
    
    ds = dataset    
    ds.cat_id = ds.cat_id.astype('Int64') # Optional when the ids are string type

    random_sample_by_classes = ds.groupby("cat_id").sample(n=images_per_class) # random_state=1 추가 가능 // frac=0.1처럼 추가 가능
    
    if specific_class != None:
        df = random_sample_by_classes.loc[ds.cat_id == specific_class]
        final_df = df.groupby("cat_id")["img_filename"]
    else:
        final_df = random_sample_by_classes.groupby("cat_id")["img_filename"]        

    for group_name, group in final_df:
        dict_for_iteration = {}
        count = 1
        while count <= images_per_class:
            key = count
            value = []
            dict_for_iteration[key] = value 
            count += 1
            
        i = 1
        for row_index, row in group.items():
            dict_for_iteration[i].append(row)
            i += 1
            
            
        for key, value in dict_for_iteration.items():
            
            img_annots = ds.loc[ds.img_filename == value[0]]
            full_image_path = str(Path(img_annots.iloc[0].img_folder, img_annots.iloc[0].img_filename))
            
            labels = []
            bboxes = []
            
            for index, row in img_annots.iterrows():
                # If there are no annotations, then skip because there are no boxes to draw
                if row["ann_bbox_xmin"] != "":
                    labels.append(f"{row['cat_id']}: {row['cat_name']}")
                    bboxes.append(
                        [
                            int(row["ann_bbox_xmin"]),
                            int(row["ann_bbox_ymin"]),
                            int(row["ann_bbox_xmax"]),
                            int(row["ann_bbox_ymax"]),
                        ]
                    )
                    
            dict_for_iteration[key].append(full_image_path)
            dict_for_iteration[key].append(labels)
            dict_for_iteration[key].append(bboxes)
                        
        image_group = []
        
        for key, value in dict_for_iteration.items():
            
            full_image_path = value[1]
            labels = value[2]
            bboxes = value[3]
            
            detected_categories = list(label.split(":")[0] for label in labels)
            np_detected_categories = np.array(detected_categories)
            
            unique_detected_categories = np.unique(np_detected_categories)
            string_unique_detected_categories = ', '.join(str(e) for e in unique_detected_categories)
            number_of_detections = len(bboxes) 
    
            img = cv2.imread(str(full_image_path))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img = cv2.putText(img, text=f"# of detections = {number_of_detections}", org=(30,30),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,0,127), thickness=2)
            img = cv2.putText(img, text=f"Categories = {string_unique_detected_categories}", org=(30,70),
                    fontFace=cv2.FONT_HERSHEY_SIMPLEX, fontScale=1, color=(255,0,127), thickness=2)
            
            image_with_boxes = img
            
            for bbox in bboxes:
                image_with_boxes = bbv.draw_rectangle(image_with_boxes, bbox, bbox_color=random.sample(range(1, 255), 3))            
            
#             img_with_boxes = bbv.draw_multiple_rectangles(img, bboxes) # bbox_color=random.sample(range(1, 255), 3)
            img_with_labels = bbv.add_multiple_labels(image_with_boxes, labels, bboxes, top=True)
            image_group.append(Image.fromarray(img_with_labels))
            
        concatenated = np.concatenate(image_group, axis=0)
        data = Image.fromarray(concatenated)
        
        print(f"Random sample for category {group_name}")
        display(data)

In [4]:
coco_json_path = '/home/ubuntu/workspace/datasets/dacon/coco/annotations/train_sample_10.json'
coco_image_path = '/home/ubuntu/workspace/datasets/dacon/coco/train'

coco_dataset = make_coco_dataset(coco_json_path, coco_image_path)

In [5]:
coco_dataset.head()

print("Dataset length : ", len(coco_dataset))

NameError: name 'coco_dataset' is not defined

In [ ]:
visualize_random_samples(coco_dataset, 4, 0)